<li>insightface : https://github.com/deepinsight/insightface/tree/master</li>
<li>project_ref : https://github.com/tuna-date/Face-Recognition-with-InsightFace</li>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from imutils import paths
import numpy as np
import argparse
import pickle
import cv2
import os

In [3]:
import os
print(os.getcwd())

/content


In [4]:
import sys

In [5]:
sys.path.append("/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/insightface/src/common")
sys.path.append("/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/insightface/deploy")

In [22]:
pip install mxnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [23]:
import face_preprocess     # /content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/insightface/src/common/face_preprocess.py

In [6]:
import face_model          # /content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/insightface/deploy/face_model.py

/usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  bool = onp.bool


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [25]:
import cv2
import numpy as np
from skimage import transform as trans

In [32]:
!pip install numpy==1.19.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 43.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)


<hr>

In [9]:
import glob

In [10]:
dataset_path = "/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets"

In [14]:
imagePaths = glob.glob(os.path.join(
    dataset_path,"**","*.*"
),recursive=True)

In [28]:
pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


In [29]:
import onnxruntime as ort
import cv2
import numpy as np
import face_preprocess

In [25]:
for i,imagePath in enumerate(imagePaths):
    # print(i,imagePath)
    print(f"processing image {i+1}/{len(imagePaths)}")
    print(imagePath.split(os.path.sep)[-2])
    break

processing image 1/1178
test


In [45]:
def create_embedding(img_path):
    embedding_model_path = '/content/drive/MyDrive/coconut/buffalo_l/w600k_r50.onnx'
    session = ort.InferenceSession(embedding_model_path)
    # img_path = "/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/train/Arya Stark/256.jpg"
    img = cv2.imread(img_path)
    img = cv2.resize(img,
                    (112,112))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = np.transpose(img,(2,0,1)).astype(np.float32)
    img = img/255.0
    img = np.expand_dims(img, axis=0)
    inputs = session.get_inputs()
    outputs = session.run(None, {inputs[0].name: img})
    face_embedding2 = outputs[0]
    # print(face_embedding2)
    return face_embedding2

In [46]:
img1 = "/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/train/Arya Stark/324.jpg"
arya1 = create_embedding(img1)

In [49]:
class FaceModel:
    def __init__(self, embedding_model_path):
        self.session = ort.InferenceSession(embedding_model_path)
    def preprocess_image(self,img_path):
        img = cv2.imread(img_path)
        # img = face_preprocess.preprocess(img,bbox,points,image_size='122,122')
        img = cv2.resize(img,
                    (112,112))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = np.transpose(img,(2,0,1)).astype(np.float32)
        img = img/255.0
        img = np.expand_dims(img, axis=0)
        return img
    def get_embedding(self,img):
        inputs = self.session.get_inputs()
        outputs = self.session.run(None,
                                   {inputs[0].name:img})
        embedding = outputs[0]
        return embedding


In [50]:
embedding_model_path = "/content/drive/MyDrive/coconut/buffalo_l/w600k_r50.onnx"
face_model = FaceModel(embedding_model_path)

In [51]:
img_path  = "/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/train/Arya Stark/324.jpg"
prep_img  = face_model.preprocess_image(img_path)
prep_img

array([[[[0.16470589, 0.16078432, 0.14901961, ..., 0.1254902 ,
          0.13333334, 0.14509805],
         [0.16862746, 0.16470589, 0.15686275, ..., 0.12156863,
          0.13333334, 0.14117648],
         [0.17254902, 0.16862746, 0.16078432, ..., 0.1254902 ,
          0.12941177, 0.13725491],
         ...,
         [0.3254902 , 0.33333334, 0.34509805, ..., 0.23921569,
          0.23921569, 0.22745098],
         [0.3372549 , 0.34509805, 0.35686275, ..., 0.22745098,
          0.22352941, 0.21568628],
         [0.31764707, 0.33333334, 0.34901962, ..., 0.21176471,
          0.19215687, 0.20392157]],

        [[0.17254902, 0.16862746, 0.15686275, ..., 0.14901961,
          0.15686275, 0.16862746],
         [0.1764706 , 0.17254902, 0.16470589, ..., 0.14509805,
          0.15686275, 0.16470589],
         [0.18039216, 0.1764706 , 0.16862746, ..., 0.13725491,
          0.15294118, 0.16078432],
         ...,
         [0.33333334, 0.34117648, 0.3529412 , ..., 0.25882354,
          0.25882354, 0.2

<hr>

In [78]:
class FaceModel:
    def __init__(self, embedding_model_path):
        self.session = ort.InferenceSession(embedding_model_path)
    def preprocess_image(self,img_path):
        img = cv2.imread(img_path)
        # img = face_preprocess.preprocess(img,bbox,points,image_size='122,122')
        img = cv2.resize(img,
                    (112,112))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = np.transpose(img,(2,0,1)).astype(np.float32)
        img = img/255.0
        img = np.expand_dims(img, axis=0)
        return img
    def get_embedding(self,img):
        inputs = self.session.get_inputs()
        outputs = self.session.run(None,
                                   {inputs[0].name:img})
        embedding = outputs[0]
        return embedding[0]

In [75]:
dataset_path = "/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets"
imagePaths = list(paths.list_images(dataset_path))
imagePaths

['/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/001.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/002.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/003.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/dogo.jpeg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/GOT.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/dogo02.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/friends.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/test/hangout_with_friends.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/train/Tywin Lannister/314.jpg',
 '/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/train/Tywin Lannister/315.jpg',
 '/content/drive/MyDrive/cocon

In [76]:
embedding_model_path = "/content/drive/MyDrive/coconut/buffalo_l/w600k_r50.onnx"
face_model = FaceModel(embedding_model_path)

In [ ]:
# img_path  = "/content/drive/MyDrive/coconut/Face-Recognition-with-InsightFace/datasets/train/Arya Stark/324.jpg"
# prep_img  = face_model.preprocess_image(img_path)
# prep_img

In [88]:
knownEmbeddings = []
knownNames = []
total = 0
for i,imagePath in enumerate(imagePaths):
    # print(f"{i} : {imagePath}")
    print(f"processing image {i+1}/{len(imagePaths)}")
    name = imagePath.split(os.path.sep)[-2]
    print(name)
    img_path = imagePath
    prep_img = face_model.preprocess_image(img_path)
    # print(prep_img)
    # Get the face embedding vector
    face_embedding = face_model.get_embedding(prep_img)
    # print(face_embedding)
    knownNames.append(name)
    knownEmbeddings.append(face_embedding)
    total = total + 1
    # break
    if i == 10:
        break
print(total," faces embedded")

data = {
    'embeddings': knownEmbeddings,
    'names' : knownNames
}

processing image 1/1161
test
processing image 2/1161
test
processing image 3/1161
test
processing image 4/1161
test
processing image 5/1161
test
processing image 6/1161
test
processing image 7/1161
test
processing image 8/1161
test
processing image 9/1161
Tywin Lannister
processing image 10/1161
Tywin Lannister
processing image 11/1161
Tywin Lannister
11  faces embedded


In [89]:
data

{'embeddings': [array([[-3.14872354e-01, -4.27063465e-01,  5.22802234e-01,
           5.14245212e-01, -4.13114160e-01, -4.71814930e-01,
          -4.64256942e-01, -2.52645075e-01, -4.69752908e-01,
           5.04767418e-01,  2.10747927e-01,  5.33535838e-01,
          -6.59637570e-01,  9.86711010e-02, -7.71820545e-01,
           3.69555384e-01,  2.80338734e-01, -3.36396486e-01,
          -9.90020558e-02, -4.32185084e-03, -3.20111364e-01,
           1.20543456e+00,  6.23659194e-01, -2.82927901e-01,
          -4.05641556e-01,  5.57939947e-01, -3.28308403e-01,
          -4.65822697e-01, -4.60866749e-01, -1.76500976e-02,
           5.54599352e-02, -1.40793115e-01,  7.50216246e-01,
           6.39490366e-01, -9.70703661e-01, -7.56544471e-02,
           2.37489015e-01,  5.63965678e-01, -1.48624510e-01,
           3.97396922e-01, -1.59988940e-01,  8.87625992e-01,
           1.09974131e-01, -3.69868308e-01, -4.19710129e-02,
          -1.88776225e-01, -4.66654181e-01, -1.11984611e-02,
          

In [95]:
import pickle

# Define the path where you want to save the embeddings
embeddings_output_path = '/content/embeddings.pickle'

# Save to output
data = {"embeddings": knownEmbeddings, "names": knownNames}


with open(embeddings_output_path, "wb") as f:
    f.write(pickle.dumps(data))

# # Alternatively, using pickle.dump for direct writing
# with open(embeddings_output_path, "wb") as f:
#     pickle.dump(data, f)